In [26]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.express as px


In [5]:
baseline = Microsimulation()



In [9]:
reform = Reform.from_dict({
  "gov.contrib.congress.wyden_smith.actc_lookback": {
    "2024-01-01.2025-12-31": True
  },
  "gov.contrib.congress.wyden_smith.per_child_actc_phase_in": {
    "2023-01-01.2025-12-31": True
  },
  "gov.irs.credits.ctc.amount.base[0].amount": {
    "2024-01-01.2025-12-31": 2100
  },
  "gov.irs.credits.ctc.refundable.individual_max": {
    "2023-01-01.2023-12-31": 1800,
    "2024-01-01.2024-12-31": 1900,
    "2025-01-01.2025-12-31": 2100
  }
}, country_id="us")
reformed = Microsimulation(reform=reform)


In [22]:
# Create a DataFrame to store all results
all_results = pd.DataFrame()

In [23]:
# Calculate for each year
for year in range(2023, 2026):
    baseline_household = baseline.calculate("household_net_income", period=year, map_to="household")
    reformed_household = reformed.calculate("household_net_income", period=year, map_to="household")
    
    # Calculate the ctc_qualifying_children at the household level
    ctc_qualifying_children = baseline.calculate("ctc_qualifying_children", period=year, map_to="household")
    
    # Get state codes at the household level
    state_codes = baseline.calculate("state_code", period=year, map_to="household")
    
    # Calculate the difference for all households
    difference = reformed_household - baseline_household
    
    # Create a DataFrame with all the results
    df_year = pd.DataFrame({
        'Year': year,
        'State': state_codes,
        'Difference': difference,
        'CTC_Qualifying_Children': ctc_qualifying_children
    })
    
    # Filter the DataFrame to households where ctc_qualifying_children >= 1
    df_filtered = df_year[df_year['CTC_Qualifying_Children'] >= 1]
    
    # Group by State and calculate the mean of the differences
    df_grouped = df_filtered.groupby('State')['Difference'].mean().reset_index()
    df_grouped['Year'] = year
    
    # Append to the all_results DataFrame
    all_results = pd.concat([all_results, df_grouped], ignore_index=True)

In [24]:
# Sort the DataFrame by State and Year
df_results = all_results.sort_values(['State', 'Year'])

# Reset the index
df_results = df_results.reset_index(drop=True)

# Rename the 'Difference' column to 'Mean_Difference' for clarity
df_results = df_results.rename(columns={'Difference': 'Mean_Difference'})

# Reorder columns
df_results = df_results[['Year', 'State', 'Mean_Difference']]

# Display the results
print(df_results)

     Year State  Mean_Difference
0    2023    AK       142.536514
1    2024    AK       289.664795
2    2025    AK       317.202393
3    2023    AL       143.732254
4    2024    AL       196.357544
..    ...   ...              ...
148  2024    WV       176.849426
149  2025    WV       308.001526
150  2023    WY        87.470161
151  2024    WY       244.454071
152  2025    WY       320.001556

[153 rows x 3 columns]


In [25]:

# Export the results to a CSV file
df_results.to_csv('state_results.csv', index=False)
print("Results have been exported to 'state_results.csv'")

Results have been exported to 'state_results.csv'


In [32]:
# Sort the DataFrame by State and Year
df_results = df_results.sort_values(['State', 'Year'])

# Generate the CSV file
csv_filename = 'state_results.csv'
df_results.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been generated.")

# Optionally, display the first few rows of the DataFrame
print("\nFirst few rows of the data:")
print(df_results.head())


CSV file 'state_results.csv' has been generated.

First few rows of the data:
   Year State  Mean_Difference
0  2023    AK       142.536514
1  2024    AK       289.664795
2  2025    AK       317.202393
3  2023    AL       143.732254
4  2024    AL       196.357544
